# Q1
Note: 
1. It is ok to define more helper functions, but not necessary for finishing the homework.
2. The imported libraries are sufficient to complete the TODO intructions. Please consult TA Vincent Cheng for permission to use additional libraries.

## Code for (a)

In [1]:
# Setup environment
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

# Build MLP
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        input_layer = 1*28*28           #784
        hidden_layer_1 = 300
        hidden_layer_2 = 100
        output_layer = 10

        self.fc1 = nn.Linear(in_features= input_layer, out_features= hidden_layer_1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(in_features= hidden_layer_1, out_features= hidden_layer_2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(in_features= hidden_layer_2, out_features= output_layer)
        
    
    def forward(self, x):

        """
        x: batch x 1 x 28 x 28
        return prediction
        """

        x = torch.flatten(x, start_dim= 1)      # batch x 784
        x = self.fc1(x)                         # batch x 300
        x = self.relu1(x)                       
        x = self.fc2(x)                         # batch x 100
        x = self.relu2(x)
        x = self.fc3(x)                         # batch x 10
        return x

mlp = MLP()
print(f'The MLP structure you built is as follow: \n{mlp}')

The MLP structure you built is as follow: 
MLP(
  (fc1): Linear(in_features=784, out_features=300, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)


## Code way of doing (b)

In [2]:
def count_parameters(model): return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters are:", count_parameters(mlp))


Total number of parameters are: 266610


## Code for (c)
Note: run this code after code for (a)

In [3]:
# helper functions

def count_acc(logits, label):
    """
    :param logits: n * c; n=num of samples, c=number of classes
    :param label: n
    :return: accuracy
    """
    pred = torch.argmax(logits, dim=1)
    return (pred == label).float().mean().item()

def train_a_model(model, num_epoch=3):
    for i_epoch in range(num_epoch):
        model.train(True)
        loss_list = [ ]
        acc_list = [ ]
        for i_iter, (x, y_true) in enumerate (train_loader, start=1):
            y_pred = model.forward(x)
            loss = F.cross_entropy(y_pred, y_true)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # for printing messages during training
            acc = count_acc(y_pred, y_true)
            loss_list.append(loss.item())
            acc_list.append(acc)
            if i_iter % 200 == 0:
                loss_avg = np.sum(loss_list) / len(loss_list)
                acc_avg = np.sum(acc_list) / len(acc_list)
                msg = "TRAIN epoch: {}/{} iter {}/{} \t loss: {:.4f}, acc: {:.2f}".format(
                    i_epoch, num_epoch, i_iter, len(train_loader), loss_avg, acc_avg*100)
                print(msg)
                loss_list = []
                acc_list = []


In [4]:
# Setup environment
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
torch.manual_seed(1)

# Prepare dataset
train_set = torchvision.datasets.MNIST("data/", train=True, transform=torchvision.transforms.ToTensor(), download=False)
test_set = torchvision.datasets.MNIST("data/", train=False, transform=torchvision.transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(train_set, batch_size= 25)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= 1000)



# Build MLP
mlp = MLP()
print(f'The MLP structure you built is as follow: \n{mlp}')

# Train MLP
lossFunc = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9)

train_a_model(mlp, num_epoch=3)



# Evaluate MLP
mlp.train(False)
mlp.eval() # toggle evaluation mode
# define an evaluation function
def evaluate_a_model(model, dataloader):
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            output = model(inputs)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).sum()
    accuracy = 100.*correct / len(dataloader.dataset)
    return accuracy
# evaluate on training data
train_acc = evaluate_a_model(mlp, train_loader)
# evaluate on training data
test_acc = evaluate_a_model(mlp, test_loader)
print(f'\nFinal training accuracy: {train_acc:.2f}%\nFinal testing accuracy: {test_acc:.2f}%')

The MLP structure you built is as follow: 
MLP(
  (fc1): Linear(in_features=784, out_features=300, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)
TRAIN epoch: 0/3 iter 200/2400 	 loss: 1.2844, acc: 66.00
TRAIN epoch: 0/3 iter 400/2400 	 loss: 0.4644, acc: 86.54
TRAIN epoch: 0/3 iter 600/2400 	 loss: 0.3898, acc: 88.26
TRAIN epoch: 0/3 iter 800/2400 	 loss: 0.3025, acc: 90.86
TRAIN epoch: 0/3 iter 1000/2400 	 loss: 0.3032, acc: 91.00
TRAIN epoch: 0/3 iter 1200/2400 	 loss: 0.2457, acc: 92.72
TRAIN epoch: 0/3 iter 1400/2400 	 loss: 0.2341, acc: 92.62
TRAIN epoch: 0/3 iter 1600/2400 	 loss: 0.2126, acc: 93.54
TRAIN epoch: 0/3 iter 1800/2400 	 loss: 0.2018, acc: 93.92
TRAIN epoch: 0/3 iter 2000/2400 	 loss: 0.2029, acc: 93.90
TRAIN epoch: 0/3 iter 2200/2400 	 loss: 0.1749, acc: 94.66
TRAIN epoch: 0/3 iter 2400/2400 	 loss: 0.1174, acc: 96.46
TRAIN epoch: 1/3 iter 200

## Code for (d)

In [5]:
# Setup environment
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

# Build Convolutional Neural Network (CNN)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
       


        self.C1 = nn.Conv2d(in_channels= 1, out_channels= 6, kernel_size= 5, padding= 2)
        self.P2 = nn.MaxPool2d(kernel_size= 2, stride=2)
        self.C3 = nn.Conv2d(in_channels= 6, out_channels= 16, kernel_size= 5, padding= 0)
        self.P4 = nn.MaxPool2d(kernel_size= 2, stride= 2)
        self.F5 = nn.Linear(in_features= 16*5*5, out_features= 120)
        self.F6 = nn.Linear(in_features= 120, out_features= 84)
        self.F7 = nn.Linear(in_features= 84, out_features= 10)

    

    def forward(self, x):
        x = F.relu(self.C1(x))
        x = self.P2(x)
        x = F.relu(self.C3(x))
        x = self.P4(x)
        x = torch.flatten(x, start_dim= 1)
        x = self.F5(x)
        x = self.F6(x)
        x = self.F7(x)

        return x

cnn = CNN()
print(f'The CNN structure you built is as follow: \n{cnn}')

The CNN structure you built is as follow: 
CNN(
  (C1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (P2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (C3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (P4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (F5): Linear(in_features=400, out_features=120, bias=True)
  (F6): Linear(in_features=120, out_features=84, bias=True)
  (F7): Linear(in_features=84, out_features=10, bias=True)
)


## Code way of doing (e)

In [7]:
print("Total number of parameters are:", count_parameters(cnn))


Total number of parameters are: 61706


## Code for (f)
Note: run this code after code for (d)

In [8]:
# Setup environment
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1)

# Prepare dataset
train_set = torchvision.datasets.MNIST("data/", train=True, transform=torchvision.transforms.ToTensor(), download=False)
test_set = torchvision.datasets.MNIST("data/", train=False, transform=torchvision.transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(train_set,batch_size=200)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=1000)


# Build Convolutional Neural Network (CNN)
cnn = CNN()
print(f'The CNN structure you built is as follow: \n{cnn}')

# Train CNN
lossFunc = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01, momentum=0.9)

#######################
# TODO: Train the CNN for 3 epochs
train_a_model(cnn, num_epoch=3)
#######################


# Evaluate CNN
cnn.eval() # toggle evaluation mode
# define an evaluation function
def evaluate_a_model(model, dataloader):
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            output = model(inputs)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).sum()
    accuracy = 100.*correct / len(dataloader.dataset)
    return accuracy
# evaluate on training data
train_acc = evaluate_a_model(cnn, train_loader)
# evaluate on training data
test_acc = evaluate_a_model(cnn, test_loader)
print(f'\nFinal training accuracy: {train_acc:.2f}%\nFinal testing accuracy: {test_acc:.2f}%')

The CNN structure you built is as follow: 
CNN(
  (C1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (P2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (C3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (P4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (F5): Linear(in_features=400, out_features=120, bias=True)
  (F6): Linear(in_features=120, out_features=84, bias=True)
  (F7): Linear(in_features=84, out_features=10, bias=True)
)
TRAIN epoch: 0/3 iter 200/300 	 loss: 1.0559, acc: 66.85
TRAIN epoch: 1/3 iter 200/300 	 loss: 0.1641, acc: 95.03
TRAIN epoch: 2/3 iter 200/300 	 loss: 0.1052, acc: 96.81

Final training accuracy: 97.19%
Final testing accuracy: 97.02%
